In [179]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
import time
import pafy
from bs4 import BeautifulSoup
import requests
import re
import webbrowser
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image
from IPython.core.display import HTML 
import time
from ipywidgets import FloatProgress
from IPython.display import display
from textblob import TextBlob
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from IPython.display import Image
from nltk.tokenize import RegexpTokenizer

In [249]:
df = pd.read_csv('consolidated.csv', index_col=0)
df2 = pd.read_csv('current_search.csv', index_col=0)

In [250]:
df = df.drop(['polarity','length','article_similarity', 'source_similarity','cosine_similarity','subject_matter','cluster'], axis =1)

In [267]:
def df_all_articles(articles_df):
    articles_df['text'] = np.nan
    for i in range(0, len(articles_df)):
        if articles_df['source_id'][i] == 'the-huffington-post':
            pass
        else:
            html_page = requests.get(articles_df['url'][i]) 
            soup = BeautifulSoup(html_page.content, 'html.parser', from_encoding="iso-8859-1")
            article = soup.findAll('p')
            str_list = []
            list_ = []
            for j in range(0,len(article)):
                clean = article[j].text
                str_list.append(clean)
            article_ = ' '.join(str_list)
            item = article_.encode("ascii", "ignore")
            clean_article = item.decode("utf-8") 
            articles_df['text'][i] = clean_article
    return articles_df

In [252]:
df_all_articles(df)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,author,content,description,publishedAt,source_id,source_name,title,url,urlToImage,text
0,https://www.facebook.com/bbcnews,Image copyright EPA Image caption Brett Kavana...,Trump's Supreme Court pick Brett Kavanaugh sai...,2018-09-05T22:25:27Z,bbc-news,BBC News,"Brett Kavanaugh, Supreme Court nominee, sidest...",https://www.bbc.co.uk/news/world-us-canada-454...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...
1,https://www.facebook.com/bbcnews,Image copyright Reuters Image caption Brett Ka...,Senator Jeff Flake says the US Supreme Court v...,2018-09-17T11:38:09Z,bbc-news,BBC News,Brett Kavanaugh: Republican senators call for ...,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...
2,https://www.facebook.com/bbcnews,Image copyright AFP/Getty Image caption Brett ...,The allegation against the US Supreme Court no...,2018-09-14T18:38:27Z,bbc-news,BBC News,Brett Kavanaugh denies sexual misconduct in hi...,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...
3,https://www.facebook.com/bbcnews,Image copyright Reuters Image caption Prof For...,Christine Blasey Ford's attorney says her clie...,2018-09-19T00:21:38Z,bbc-news,BBC News,Christine Blasey Ford: Kavanaugh accuser 'face...,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...
4,Sabrina Siddiqui in Washington,"Brett Kavanaugh, Donald Trump’s conservative n...",Trump’s supreme court nominee was grilled on a...,2018-09-05T18:14:46Z,the-guardian-au,The Guardian (AU),Brett Kavanaugh sidesteps Senate questions on ...,https://www.theguardian.com/us-news/2018/sep/0...,https://i.guim.co.uk/img/media/e57aaa3036c2594...,Trumps supreme court nominee was grilled on ab...
5,https://www.facebook.com/bbcnews,The Senate has scheduled a public hearing on a...,The Supreme Court nominee and his accuser are ...,2018-09-17T22:30:43Z,bbc-news,BBC News,Kavanaugh and accuser to testify in Senate,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...
6,Lauren Gambino in Washington,NaN,"Donald Trump’s pick, who could shift the top d...",2018-09-04T14:03:45Z,the-guardian-au,The Guardian (AU),Brett Kavanaugh: protesters disrupt supreme co...,https://www.theguardian.com/law/live/2018/sep/...,https://i.guim.co.uk/img/media/23b642bbb668dec...,White House claims security intervened before ...
7,"acollman@businessinsider.com (Ashley Collman),...",Sen. Susan Collins says Brett Kavanaugh told h...,Reuters Sen. Susan Collins of Maine met with S...,2018-08-21T18:37:00Z,business-insider,Business Insider,Sen. Susan Collins says Supreme Court nominee ...,https://www.businessinsider.com/r-senator-coll...,https://amp.businessinsider.com/images/5b7c57d...,createPerformanceMark('first image displayed')...
8,Bryan Logan,Democratic senators kept the heat on Judge Bre...,Judge Brett Kavanaugh on the third day of his ...,2018-09-07T04:19:34Z,business-insider,Business Insider,Brett Kavanaugh ducked questions about whether...,https://www.businessinsider.com/brett-kavanaug...,https://amp.businessinsider.com/images/5b91fbe...,Democratic senators kept the heat on Judge B...
9,Associated Press,WASHINGTON (AP) — Senate Democrats worked into...,Andrew Harnik/AP After two marathon days in th...,2018-09-07T09:45:05Z,business-insider,Business Insider,"Kavanaugh looks polished, set for confirmation...",https://www.businessinsider.com/kavanaugh-look...,https://amp.businessinsider.com/images/5b92484...,WASHINGTON (AP) Senate Democrats worked int...


In [266]:
list(df.text)[-2]

'Advertisement Supported by Best of Late Night By Giovanni Russonello Welcome to Best of Late Night, a rundown of the previous nights highlights that lets you sleep  and lets us get paid to watch comedy. If youre interested in hearing from The Times regularly about great TV, sign up for our Watching newsletter and get recommendations straight to your inbox. An allegation of sexual assault has thrown the Supreme Court confirmation of Judge Brett Kavanaugh into jeopardy, with the accuser seeking an F.B.I. investigation and Senator John Kennedy, a Louisiana Republican, saying her charge had turned the process into an intergalactic freak show. On The Late Late Show on Tuesday, James Corden said he didnt see things that way. An intergalactic freak show. I like to think that billions of light-years away, an emperor of some alien galaxy is really offended that someone just compared him to Brett Kavanaugh.  JAMES CORDEN Republicans have also been critical of the timing of the misconduct allega

In [254]:
df = df.dropna()

In [255]:
df = df.reset_index()
df = df.drop('index',axis = 1)

In [259]:
# df = df.drop('level_0',axis = 1)

In [260]:
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

def df_with_polarity_score(art_df):
    art_df['neg'] = np.nan
    art_df['neu'] = np.nan
    art_df['pos'] = np.nan
    art_df['compound'] = np.nan
    art_df = art_df.reset_index(drop=True)
    for i in range(0, len(art_df)):
        scores = sid.polarity_scores(art_df['text'][i])
        art_df['neg'][i] = scores['neg']
        art_df['neu'][i] = scores['neu']
        art_df['pos'][i] = scores['pos']
        art_df['compound'][i] = scores['compound']
    return art_df

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/markrethana/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [243]:
# from sklearn.feature_extraction.text import CountVectorizer
# cvec = CountVectorizer()
# cvec.fit(my_df.text)
# neg_doc_matrix = cvec.transform(my_df[my_df.target == 0].text)
# pos_doc_matrix = cvec.transform(my_df[my_df.target == 1].text)
# neg_tf = np.sum(neg_doc_matrix,axis=0)
# pos_tf = np.sum(pos_doc_matrix,axis=0)
# neg = np.squeeze(np.asarray(neg_tf))
# pos = np.squeeze(np.asarray(pos_tf))
# term_freq_df = pd.DataFrame([neg,pos],columns=cvec.get_feature_names()).transpose()

In [261]:
def df_with_subjectivity(df):
    df['subjectivity'] = np.nan
    for i in range(0, len(df)):
        analysis = TextBlob((df['text'][i]))
        df['subjectivity'][i] =  analysis.sentiment.subjectivity
    return df
                            

In [268]:
final_df = df_with_polarity_score(df)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set 

In [269]:
final_df = df_with_subjectivity(final_df)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [270]:
final_df

,author,content,description,publishedAt,source_id,source_name,title,url,urlToImage,text,neg,neu,pos,compound,subjectivity
0,https://www.facebook.com/bbcnews,Image copyright EPA Image caption Brett Kavana...,Trump's Supreme Court pick Brett Kavanaugh sai...,2018-09-05T22:25:27Z,bbc-news,BBC News,"Brett Kavanaugh, Supreme Court nominee, sidest...",https://www.bbc.co.uk/news/world-us-canada-454...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...,0.073,0.818,0.109,0.9753,0.444473
1,https://www.facebook.com/bbcnews,Image copyright Reuters Image caption Brett Ka...,Senator Jeff Flake says the US Supreme Court v...,2018-09-17T11:38:09Z,bbc-news,BBC News,Brett Kavanaugh: Republican senators call for ...,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...,0.056,0.899,0.046,-0.3420,0.486057
2,https://www.facebook.com/bbcnews,Image copyright AFP/Getty Image caption Brett ...,The allegation against the US Supreme Court no...,2018-09-14T18:38:27Z,bbc-news,BBC News,Brett Kavanaugh denies sexual misconduct in hi...,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...,0.042,0.858,0.100,0.9750,0.428211
3,https://www.facebook.com/bbcnews,Image copyright Reuters Image caption Prof For...,Christine Blasey Ford's attorney says her clie...,2018-09-19T00:21:38Z,bbc-news,BBC News,Christine Blasey Ford: Kavanaugh accuser 'face...,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...,0.073,0.829,0.098,0.9776,0.444317
4,Sabrina Siddiqui in Washington,"Brett Kavanaugh, Donald Trump’s conservative n...",Trump’s supreme court nominee was grilled on a...,2018-09-05T18:14:46Z,the-guardian-au,The Guardian (AU),Brett Kavanaugh sidesteps Senate questions on ...,https://www.theguardian.com/us-news/2018/sep/0...,https://i.guim.co.uk/img/media/e57aaa3036c2594...,Trumps supreme court nominee was grilled on ab...,0.040,0.840,0.120,0.9977,0.426633
5,https://www.facebook.com/bbcnews,The Senate has scheduled a public hearing on a...,The Supreme Court nominee and his accuser are ...,2018-09-17T22:30:43Z,bbc-news,BBC News,Kavanaugh and accuser to testify in Senate,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...,0.066,0.867,0.067,0.2703,0.395146
6,"acollman@businessinsider.com (Ashley Collman),...",Sen. Susan Collins says Brett Kavanaugh told h...,Reuters Sen. Susan Collins of Maine met with S...,2018-08-21T18:37:00Z,business-insider,Business Insider,Sen. Susan Collins says Supreme Court nominee ...,https://www.businessinsider.com/r-senator-coll...,https://amp.businessinsider.com/images/5b7c57d...,createPerformanceMark('first image displayed')...,0.055,0.877,0.068,0.7786,0.406409
7,Bryan Logan,Democratic senators kept the heat on Judge Bre...,Judge Brett Kavanaugh on the third day of his ...,2018-09-07T04:19:34Z,business-insider,Business Insider,Brett Kavanaugh ducked questions about whether...,https://www.businessinsider.com/brett-kavanaug...,https://amp.businessinsider.com/images/5b91fbe...,Democratic senators kept the heat on Judge B...,0.024,0.891,0.084,0.9639,0.445306
8,Associated Press,WASHINGTON (AP) — Senate Democrats worked into...,Andrew Harnik/AP After two marathon days in th...,2018-09-07T09:45:05Z,business-insider,Business Insider,"Kavanaugh looks polished, set for confirmation...",https://www.businessinsider.com/kavanaugh-look...,https://amp.businessinsider.com/images/5b92484...,WASHINGTON (AP) Senate Democrats worked int...,0.068,0.839,0.093,0.9773,0.461014
9,Joe Perticone,WASHINGTON — Days after allegations surfaced t...,Supreme Court nominee Brett Kavanaugh has come...,2018-09-17T17:34:20Z,business-insider,Business Insider,B

In [253]:
def get_vocab(df):
    tokenizer = RegexpTokenizer('[a-z]\w+')
    all_text = []
    for blob in df.text:
        new_blob = blob.lower()
        all_text.append(new_blob)
    corpus = '-'.join(all_text)
#     corpus = corpus.encode("ascii", "ignore")
#     print(str(corpus))
    token = tokenizer.tokenize(corpus)
    encoded_tokens = []
#     for item in token:
#         item = item.encode("ascii", "ignore")
#         item = item.decode("utf-8") 
#         encoded_tokens.append(item)
    vocab = sorted(set(encoded_token))
    return vocab

In [234]:
unicode_string = u"hello aåbäcö"
str(unicode_string.encode("ascii", "ignore"))


"b'hello abc'"

In [235]:
new_tokenized = get_vocab(final_df)

In [236]:
def word_metrics(vocab):
    all_metrics = []
    for word in vocab:
        analysis = TextBlob(word)
        metric = {'word':word,'subjectivity':analysis.sentiment.subjectivity,'polarity':analysis.sentiment.polarity}
        all_metrics.append(metric)
    return pd.DataFrame(all_metrics)

In [237]:
word_df = word_metrics(new_tokenized)

In [238]:
word_df

,polarity,subjectivity,word
0,0.0000,0.000,a
1,0.0000,0.000,aalayah
2,0.0000,0.000,abandon
3,0.0000,0.000,abandoning
4,0.0000,0.000,abc
5,0.0000,0.000,abdicating
6,0.0000,0.000,abel
7,0.0000,0.000,abetting
8,0.0000,0.000,abeyance
9,-0.7000,0.800,abhorrent


In [136]:
word_df.to_csv('word_metrics.csv')

In [173]:
all_words = pd.read_csv('word_metrics.csv')
all_words['word']= all_words['word'].astype(str)

def word_cloud(source_name,subjectivity_floor):
    
    list_words = all_words.word[(all_words.subjectivity < subjectivity_floor)]
    df2 = final_df[(final_df.source_name == source_name)]
    all_text = []
    for blob in df2.text:
        all_text.append(blob)
    corpus = '-'.join(all_text)
    corpus = corpus.lower()
    stopwords = set(STOPWORDS)
    stopwords.update(list_words)

    # Generate a word cloud image
    wordcloud = WordCloud(background_color="white", stopwords=stopwords).generate(corpus)
    # Display the generated image:
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
html_page = requests.get('https://www.youtube.com/results?search_query='+split[0]+'+'+'+'+split[1])
soup = BeautifulSoup(html_page.content, 'html.parser', from_encoding='utf-8')
videos = soup.findAll('a')

In [95]:
final_df

,author,content,description,publishedAt,source_id,source_name,title,url,urlToImage,text,neg,neu,pos,compound,subjectivity
0,https://www.facebook.com/bbcnews,Image copyright EPA Image caption Brett Kavana...,Trump's Supreme Court pick Brett Kavanaugh sai...,2018-09-05T22:25:27Z,bbc-news,BBC News,"Brett Kavanaugh, Supreme Court nominee, sidest...",https://www.bbc.co.uk/news/world-us-canada-454...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...,0.080,0.821,0.099,0.8899,0.425482
1,https://www.facebook.com/bbcnews,Image copyright Reuters Image caption Brett Ka...,Senator Jeff Flake says the US Supreme Court v...,2018-09-17T11:38:09Z,bbc-news,BBC News,Brett Kavanaugh: Republican senators call for ...,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...,0.061,0.901,0.038,-0.9428,0.473246
2,https://www.facebook.com/bbcnews,Image copyright AFP/Getty Image caption Brett ...,The allegation against the US Supreme Court no...,2018-09-14T18:38:27Z,bbc-news,BBC News,Brett Kavanaugh denies sexual misconduct in hi...,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...,0.052,0.864,0.084,0.9353,0.403486
3,https://www.facebook.com/bbcnews,Image copyright Reuters Image caption Prof For...,Christine Blasey Ford's attorney says her clie...,2018-09-19T00:21:38Z,bbc-news,BBC News,Christine Blasey Ford: Kavanaugh accuser 'face...,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...,0.079,0.831,0.089,0.9226,0.425316
4,Sabrina Siddiqui in Washington,"Brett Kavanaugh, Donald Trump’s conservative n...",Trump’s supreme court nominee was grilled on a...,2018-09-05T18:14:46Z,the-guardian-au,The Guardian (AU),Brett Kavanaugh sidesteps Senate questions on ...,https://www.theguardian.com/us-news/2018/sep/0...,https://i.guim.co.uk/img/media/e57aaa3036c2594...,Trumpâs supreme court nominee was grilled on...,0.038,0.846,0.115,0.9976,0.420556
5,https://www.facebook.com/bbcnews,The Senate has scheduled a public hearing on a...,The Supreme Court nominee and his accuser are ...,2018-09-17T22:30:43Z,bbc-news,BBC News,Kavanaugh and accuser to testify in Senate,https://www.bbc.co.uk/news/world-us-canada-455...,https://ichef.bbci.co.uk/news/1024/branded_new...,Share this with Email Facebook Messenger Mess...,0.071,0.870,0.059,-0.8260,0.377921
6,"acollman@businessinsider.com (Ashley Collman),...",Sen. Susan Collins says Brett Kavanaugh told h...,Reuters Sen. Susan Collins of Maine met with S...,2018-08-21T18:37:00Z,business-insider,Business Insider,Sen. Susan Collins says Supreme Court nominee ...,https://www.businessinsider.com/r-senator-coll...,https://amp.businessinsider.com/images/5b7c57d...,createPerformanceMark('first image displayed')...,0.055,0.877,0.068,0.7786,0.406409
7,Bryan Logan,Democratic senators kept the heat on Judge Bre...,Judge Brett Kavanaugh on the third day of his ...,2018-09-07T04:19:34Z,business-insider,Business Insider,Brett Kavanaugh ducked questions about whether...,https://www.businessinsider.com/brett-kavanaug...,https://amp.businessinsider.com/images/5b91fbe...,Democratic senators kept the heat on Judge B...,0.024,0.891,0.084,0.9639,0.445306
8,Associated Press,WASHINGTON (AP) — Senate Democrats worked into...,Andrew Harnik/AP After two marathon days in th...,2018-09-07T09:45:05Z,business-insider,Business Insider,"Kavanaugh looks polished, set for confirmation...",https://www.businessinsider.com/kavanaugh-look...,https://amp.businessinsider.com/images/5b92484...,WASHINGTON (AP) — Senate Democrats worked in...,0.068,0.839,0.093,0.9773,0.461014
9,Joe Perticone,WASHINGTON — Days after allegations surfaced t...,Supreme Court nominee Brett Kavanaugh has come...,2018-09-17T17:34:20Z,business-insider,Business Insider

In [96]:
final_df.to_csv('bk_corpus.csv')